### Augmentation

Images are easier to manipulate than text.
If I take an image I can make it a bit darker and it is still an image of the same thing.
There are many different ways I can change an image without breaking it.

A nice set of transforms can be viewed [in the fastai docs](https://docs.fast.ai/vision.transform.html#List-of-transforms).

It's desirable to randomly apply these transformations, to increase the range of images that can be produced.

In [ ]:
!mkdir -p massive-data/cat
!wget https://github.com/matthewfranglen/interactive-image-classification/raw/master/notebooks/massive-data/cat/cat.jpg -O massive-data/cat/cat.jpg

In [ ]:
from typing import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim

import numpy as np
from torchvision import datasets, transforms

from tqdm import tqdm

import PIL

In [ ]:
def to_image(image: torch.Tensor) -> PIL.Image:
    # the rescaling also reverses the normalization (close enough)
    image -= image.min()
    image /= image.max()
    return transforms.functional.to_pil_image(image.cpu(), 'RGB')

In [ ]:
image = PIL.Image.open("massive-data/cat/cat.jpg") ; image

In [ ]:
transforms.RandomHorizontalFlip(p=1)(image)

In [ ]:
transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5)(image)

In [ ]:
transforms.RandomAffine(degrees=90.)(image)

These augmentations are applied to the image before it is transformed into a tensor.
This is because it's much easier to apply these changes using the python image library instead of operating over the tensor.

In [ ]:
image.rotate(angle=45)

In [ ]:
train_ds = datasets.DatasetFolder(
    root='massive-data',
    loader=PIL.Image.open,
    extensions=('jpg',),
    transform=transforms.Compose([
        # This lets you randomly apply all the transformations in this list.
        # The test is not once per transform, it either skips all transforms or applies all transforms.
        transforms.RandomApply([
            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
        ]),
        
        # This is a combination of RandomApply and HorizontalFlip, by default has a 50% chance of flipping the image
        transforms.RandomHorizontalFlip(),

        transforms.ToTensor()
    ])
)

image, target = next(iter(train_ds))
print(train_ds.classes[target])
to_image(image)